In [ ]:
import os
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import plotly.offline as py
py.init_notebook_mode()

In [ ]:
from dautil.plot import iplot_column_slider, plot_column_slider

In [ ]:
%matplotlib inline

# Power Spectra

In [ ]:
def extract_z(path):
    '''`path`: example: ps_z029.91_nf0.999799_useTs1_aveTb-03.79_200_300Mpc_v3
    '''
    return float(path.name.split('_')[1][1:])

In [ ]:
def read_txt(path):
    df = pd.read_csv(path, delim_whitespace=True, header=None, names=('k', 'P_k', 'std'))
    df.set_index('k', inplace=True)
    return df

In [ ]:
def read_txts(paths):
    dfs = (read_txt(path) for path in paths)
    zs = [extract_z(path) for path in paths]
    df = pd.concat(dfs, keys=zs)
    df.index.names = ('z', 'k')
    return df

In [ ]:
%time df = read_txts(list(Path('~/21cmfast/21cmFAST/Out-of-box_Output/Power_spectra').expanduser().glob('*')))

In [ ]:
%time df1 = read_txts(list(Path('~/21cmfast/21cmFAST/Output_files/Deldel_T_power_spec').expanduser().glob('*')))

In [ ]:
def reshape_P_k(df):
    return (df.P_k + 1.j * df['std']).unstack().T

In [ ]:
P_k = reshape_P_k(df)

In [ ]:
P_k1 = reshape_P_k(df1)

In [ ]:
P_k_com = P_k.apply(np.real) + 1.j * P_k1.apply(np.real)

In [ ]:
P_k.head()

In [ ]:
%%opts Curve {+framewise} [logx=True logy=True]
plot_column_slider(P_k1)

In [ ]:
%%opts Curve {+framewise}
# [logx=True logy=True]
plot_column_slider(P_k_com)

In [ ]:
py.iplot(iplot_column_slider(P_k1.apply(np.real)))

In [ ]:
col_wo_na = P_k1.columns[~np.any(P_k1.isna(), axis=0)].values

In [ ]:
col_wo_na.sort()

In [ ]:
P_k1 = P_k1[col_wo_na]

# Global Evolution

In [ ]:
def read_global_evolution(path):
    df = pd.read_csv(path, delim_whitespace=True, header=None, names=('zp', 'filling_factor_of_HI_zp', 'Tk_ave', 'x_e_ave', 'Ts_ave', 'T_cmb*(1+zp)', 'J_alpha_ave', 'xalpha_ave', 'Xheat_ave', 'Xion_ave'))
    df.set_index('zp', inplace=True)
    return df

In [ ]:
path = Path('~/21cmfast/21cmFAST/Output_files/Ts_outs/global_evolution_Nsteps40_zprimestepfactor1.020_L_X3.2e+40_alphaX1.0_f_star100.0500_alpha_star0.5000_f_esc100.1000_alpha_esc-0.5000_Mturn5.0e+08_t_star0.5000_Pop2_200_300Mpc')

In [ ]:
path = Path('~//21cmfast/21cmFAST/Output_files/Ts_outs/global_evolution_Nsteps40_zprimestepfactor1.020_L_X3.2e+40_alphaX1.0_f_star100.0500_alpha_star0.5000_f_esc100.1000_alpha_esc-0.5000_Mturn5.0e+08_t_star0.5000_Pop2_200_300Mpc')

In [ ]:
!tail $path

In [ ]:
df = read_global_evolution(path)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.head()

In [ ]:
df.index.values

In [ ]:
df_big_z = df.loc[df.index >= 5.9]
# df_big_z = df

In [ ]:
df.shape

In [ ]:
py.iplot(iplot_column_slider(df))

In [ ]:
def format_float_array(array):
    string = ','.join(f'{i:.16}' for i in array)
    print(len(string))
    return string

In [ ]:
df_big_z.shape[0] + 3

In [ ]:
print(','.join(('0', '5.95', format_float_array(df_big_z.index.values), '30.')))

In [ ]:
print(','.join(('-2', '-1', format_float_array(df_big_z.x_e_ave.values), '0')))